### Import and Load Images

In [1]:
import numpy as np
import cv2
import  matplotlib.pyplot as plt
import os
import sys
sys.path.append('../Scripts/')
from Scripts.Utility import Utility2
ut = Utility2()

root_dir = os.path.abspath("../")
imagePath = "/Input/Images"

in init


In [2]:
images = ut.loadImages(imagePath)
#ut.showImages(images, scale=(500,500))
#transmitted, channels = ut.loadImages(imagePath)
#ut.showImages(transmitted, scale=(500,500))
#ut.showImages(channels, scale=(500,500))

### Apply processing
1. Gabor
2. Histogram Equalization
3. Median Smooth


![Histogram Comparison](/ImageWrite/histogramComparison2.png "Image")

In [9]:
processOutput = []
processOutputCLAHEeq = []
processOutputBaseEq = []
for image in images:
    smoothed_eq_img_arr, smt_arr, f_img, f_values = ut.processImage(image)

    processOutput.append(smoothed_eq_img_arr)


    grayImg = ut.grayscaleImages([image])
    baseeq = ut.grayscaleImages([ut.histogram_equalization(image)])

    processOutputBaseEq.append(baseeq[0])

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    mg_eq = clahe.apply(grayImg[0])

    processOutputCLAHEeq.append(mg_eq)



#smoothed_eq_img_arr, smt_arr, f_img, f_values = ut.processImage(images[0])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--BFP.tif")
#smoothed_eq_img_arr1, smt_arr1, f_img1, f_values1 = ut.processImage(images[1])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--DsRed.tif")
#smoothed_eq_img_arr2, smt_arr2, f_img2, f_values2 = ut.processImage(images[2])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--eGFP.tif")
#smoothed_eq_img_arr3, smt_arr3, f_img3, f_values3 = ut.processImage(images[3])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--miRFP.tif")


pairs = ut.pairs_from_array(processOutputCLAHEeq, processOutputBaseEq)
#pairs = ut.pairs_from_array(ut.grayscaleImages(images), pairs)

ut.showImages(pairs, scale=(1500,500))
#ut.writeImages(pairs, path="../ImageWrite", prefix="histogramComparison", filetype=".png", scale=(500,250))

### Apply Median Smoothing

In [7]:
medianSmoothed = ut.medianBlurImages(processOutputCLAHEeq)
ut.showImages(medianSmoothed, scale=(500,500))

### Merge Processed Images (Single Pair)
Gamma = 0.25

Theta = pi/4 && 3*pi/4

From index 1 and 5 of processed images

In [8]:
#smts = [smoothed_eq_img_arr,smoothed_eq_img_arr1, smoothed_eq_img_arr2,smoothed_eq_img_arr3]
#mergedImages = ut.mergeImages(smts, 0,3, 0.5)
mergedImages = []

for x in processOutput:
    mergedImage = cv2.addWeighted(x[1], 0.5, x[5], .5, 0)
    mergedImages.append(mergedImage)
    
#ut.showImages(mergedImages, scale=(500,500))

pairs = ut.pairs_from_array(processOutputCLAHEeq, mergedImages)
pairs = ut.pairs_from_array(ut.grayscaleImages(images), pairs)

ut.showImages(pairs, scale=(1500,500))


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\matrix_operations.cpp:68: error: (-215:Assertion failed) src[i].dims <= 2 && src[i].rows == src[0].rows && src[i].type() == src[0].type() in function 'cv::hconcat'


### Watershed Process

In [6]:
#mergedImages = [mergedImage, mergedImage1, mergedImage2, mergedImage3]
#GrayMergedArr = ut.grayscaleImages(mergedImages)

ThresholdArr = ut.thresholdImages(mergedImages)
#ut.showImages(ThresholdArr, scale=(500,500))
kmeansApplied = ut.kmeansOutImages(ThresholdArr, 5)



ut.showImages(kmeansApplied, scale=(500,500))

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\imgproc\src\thresh.cpp:1557: error: (-2:Unspecified error) in function 'double __cdecl cv::threshold(const class cv::_InputArray &,const class cv::_OutputArray &,double,double,int)'
> THRESH_OTSU mode:
>     'src_type == CV_8UC1 || src_type == CV_16UC1'
> where
>     'src_type' is 16 (CV_8UC3)


In [ ]:
ThresholdedKmeans = []
for x in range(len(kmeansApplied)):
    ret, thresh = cv2.threshold(kmeansApplied[x], 191, 255, cv2.THRESH_BINARY)
    ThresholdedKmeans.append(thresh)


#GrayScaleKmeans = ut.grayscaleImages(kmeansApplied)
ThresholdedKmeans.pop()
ut.showImages(ThresholdedKmeans, scale=(500,500))

singleImage = ut.pairMerge(ThresholdedKmeans)

ut.showImages([singleImage], scale=(500,500))

#histSingleImage  = ut.histogram_equalization(singleImage)

#ut.showImages([histSingleImage,singleImage], scale=(500,500))

#ut.writeImages([histSingleImage,singleImage])
ret, thresh = cv2.threshold(singleImage, 128, 255, cv2.THRESH_BINARY)

ut.showImages([singleImage,thresh], scale=(500,500))

#ut.writeImages([singleImage,thresh],"postMerge", ["singleImage" , "thresh"] )

#ThresholdedKmeans.append(thresh)


print(np.shape(kmeansApplied))
#pairs = ut.pairs_from_array(ThresholdedKmeans, kmeansApplied)
#ut.showImages(pairs, scale=(500,500))

#output, markers = ut.waterShedImages(kmeansApplied)


#ut.showImages(output, scale=(500,500))
